In [24]:
# 🔧 Setup
# ----------------------------------------
!pip install pathway bokeh panel --quiet

In [25]:
import pandas as pd
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import time
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

pn.extension()

In [26]:
#Upload your dataset File
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset (1).csv


In [27]:
df = pd.read_csv('dataset.csv')

In [28]:
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity']].to_csv('parking_stream.csv', index=False)

In [29]:
#🧱 Define Pathway schema
# ----------------------------------------
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int

In [30]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=9000)

In [31]:
# 🧠 Price calculation setup
# ----------------------------------------
BASE_PRICE = 10.0
ALPHA = 2.0

# Parse + enrich + filter
data_clean = data.with_columns(
    lot_id = data.SystemCodeNumber,
    timestamp_parsed = data.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    occ = data.Occupancy,
    cap = data.Capacity
).filter(
    (pw.this.occ.is_not_none()) &
    (pw.this.cap.is_not_none()) &
    (pw.this.timestamp_parsed.is_not_none())
)

In [54]:
micro = (
    data_clean.groupby(pw.this.lot_id)
    .windowby(
        pw.this.timestamp_parsed,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30))
    )
    .reduce(
        lot_id=pw.reducers.any(pw.this.lot_id),  # ✅ CORRECTED LINE
        occ_sum=pw.reducers.sum(pw.this.occ),
        occ_count=pw.reducers.count(1),
        cap_max=pw.reducers.max(pw.this.cap),
        t=pw.reducers.max(pw.this.timestamp_parsed)
    )
    .with_columns(
        price = BASE_PRICE + ALPHA * ((pw.this.occ_sum / pw.this.occ_count) / pw.this.cap_max)
    )

)


In [55]:
# Panel Live Plotting
def model1_plotter(source):
    fig = figure(
        height=300, width=700,
        title="Real-Time Price Stream", x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.circle("t", "price", source=source, size=6, color="black")
    return fig

# Create a plot per parking lot
plots = []
for lot in df['SystemCodeNumber'].unique():
    lot_table = micro.filter(pw.this.lot_id == lot)
    viz = lot_table.plot(model1_plotter, sorting_col="t")
    plots.append(pn.Column(f"## {lot}", viz))

# Display all 14 plots
pn.Column(*plots).servable()


Column
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [3] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [4] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [5] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [6] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [7] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [8] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [9] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [10] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [11] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [12] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [13] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)

In [56]:
%%capture --no-display
pw.run()


Output()

In [50]:
# 💾 Clean CSV write (overwrite mode)
pw.io.csv.write(micro, "model1_output.csv")  # ✅ Valid


In [51]:
# ▶️ Run pipeline in background
# ----------------------------------------
%%capture --no-display
pw.run()

Output()